# Maximize macroscopic entropy production rate for the ABC Model

![ABC-model](Metabolic-network.JPG "ABC Metabolic network")

## All Constraints

$$ \begin{array}{ll}
    \underset{\vec{\bf r}_+,\vec{\bf r}_-,\vec{\bf c}}{\mbox{maximize}}   & - \sum_j\left({\mathscr P}_{+j}\log({\mathscr P}_{+j}) + {\mathscr P}_{-j}\log({\mathscr P}_{-j})\right)\cdot v_{growth}   \\
    \mbox{subject to}  & S\cdot(\vec{\bf L}_+-\vec{\bf L}_-) = 0  & \text{Steady-state Net Likelihood Constraint} \\
                         & S\cdot(\vec{\bf r}_+-\vec{\bf r}_-) = 0 & \text{Steady-state net flux Constraint} \\
                       &\log\vec{\bf r}_+ -\log\vec{\bf r}_-= -\frac{1}{RT}S^T\cdot\vec{\mu}^0 - S^T\cdot\log\vec{\bf c} & \text{Thermodynamic Constraint} \\
                      %% & \log\vec{\bf L}_- = \frac{1}{RT}S^T\cdot\vec{\mu}^0 + S^T\cdot\log\vec{\bf c} \\
                      %% & \log\vec{\bf L}_+  -\log\vec{\bf L}_- \\
                     %% & \vec{v}_{lower}(L_+) \leq \vec{\bf r}_+ -\vec{\bf r}_- \leq \vec{v}_{upper}(L_+) \\
                         & \vec{\bf r}_+ \geq 0 & \text{Positive rate Constraint} \\
                         & \vec{\bf r}_- \geq 0 & \text{Positive rate Constraint} \\
                      & \log \left[{\bf A}_{ext}\right] = \log \left[{\bf E}_{ext}\right] =  \log c_{upper} & \text{Input boundary Constraint} \\
                      & \log \left[{\bf E}_{ext}\right] = \log \left[{\bf D}_{ext}\right] = \log c_{lower}  & \text{Output boundary Constraint} \\         
      & -\left|\Delta G_{ext}\right| \leq S^T\mu \leq \left|\Delta G_{ext}\right|
    \end{array} 
$$

Where
* $\vec{\bf r}_+,\vec{\bf r}_-$ are decision variables representing the forward and backward rates, respectively.
* ${\mathscr P}_{+i},{\mathscr P}_{-i}$ are the normalized forward and backward thermodynamic driving forces $\frac{r_{+i}}{r_{-i}}\left(\sum_j\frac{r_{+j}}{r_{-j}} + \frac{r_{-j}}{r_{+j}}\right)^{-1}$ and $\frac{r_{-i}}{r_{+i}}\left(\sum_j\frac{r_{+j}}{r_{-j}} + \frac{r_{-j}}{r_{+j}}\right)^{-1}$
* $\vec{\bf c}$ is a decision variable representing the chemical concentrations of each metabolite 
*  $S$ is the $m\times n$ stoichiometric matrix of representing $m$ metabolites and $n$ reactions of the model  
*  $\vec{\mu}^0$ is the vector of standard chemical potentials 
* $L_+ = \frac{r_+}{r_-}$
* $L_- = \frac{r_-}{r_+}$
* $v_{lower}(L) =\min\left(-1,\text{sign}(\log L)\cdot L^{\text{sign}(\log L)}\right) + 1$
* $v_{upper}(L) = \max\left(1,\text{sign}(\log L)\cdot L^{\text{sign}(\log L)}\right) - 1$
* $\Delta G_{ext} =  \mu_{D_{ext}} + \mu_{F_{ext}} - \mu_{A_{ext}} - \mu_{E_{ext}}$

Decision variables are emphasized in **bold**. 

In [3]:
import mentos
import pandas as pd, os, sys
import numpy as np
from mentos.abc_model import fullS, S, deltaG0, mu0, \
     c_U, c_L, mets, external_mets, met_bounds, internal_mets, rxns, \
     efflux, uptake, n, m, R, T, biomass, \
     external_free_energy, efflux, uptake, A_ext, D_ext, E_ext, F_ext, \
     v_L, v_U
from scipy.stats import entropy
from IPython.display import display, Markdown, Latex, SVG, Image, HTML
abc_dir = ''
metab, reactions = {}, {}

convex_mentos = 'convex_mentos_likelihoods_first_elmo'
metab[convex_mentos] = pd.read_table(os.path.join(convex_mentos, 'ABC_metab_{}.csv'.format(convex_mentos)), index_col=0)
#display(metab[convex_mentos])

boltzmann = 'boltzmann'
metab[boltzmann] = pd.read_table(os.path.join(boltzmann, 'ABC_metab_{}.csv'.format(boltzmann)), index_col=0)
#display(metab[convex_mentos])

reactions[convex_mentos] = pd.read_table(os.path.join(convex_mentos, 'ABC_reaction_{}.csv'.format(convex_mentos)), index_col=0)
reactions[boltzmann] = pd.read_table(os.path.join(boltzmann, 'ABC_reaction_{}.csv'.format(boltzmann)), index_col=0)
#display(reactions[convex_mentos])

m,n = fullS.shape
i,n = S.shape

def macroscopic_entropy_production_rate_and_net_likelihood_ss( x, fullS, mu0, deltaG0, R, T ):
    log_c, \
    forward_rate, backward_rate, \
    log_Q, log_K, \
    forward_likelihood, backward_likelihood, \
    forward_probability, backward_probability, mu, \
    thermodynamic_driving_force, \
    net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
    return entropy( np.concatenate( ( forward_probability, backward_probability ) ) )*net_flux[biomass] - np.sum(np.square(np.dot(S,forward_likelihood - backward_likelihood)))

def constraints_on_likelihoods( x, fullS, mu0, deltaG0, R, T ):
    log_c, \
    forward_rate, backward_rate, \
    log_Q, log_K, \
    forward_likelihood, backward_likelihood, \
    forward_probability, backward_probability, mu, \
    thermodynamic_driving_force, \
    net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
    return np.concatenate((np.log(forward_likelihood)  + log_Q - log_K, # = 0
                           [  log_c[A_ext] - np.log(c_U),     # = 0               # Creating a concentration gradient between A, E and D,F
                              log_c[E_ext] - np.log(c_U),     # = 0
                              log_c[D_ext] - np.log(c_L),     # = 0
                              log_c[F_ext] - np.log(c_L) ]))  #= 0 

def macroscopic_entropy_production_rate( x, fullS, mu0, deltaG0, R, T ):
    log_c, \
    forward_rate, backward_rate, \
    log_Q, log_K, \
    forward_likelihood, backward_likelihood, \
    forward_probability, backward_probability, mu, \
    thermodynamic_driving_force, \
    net_flux = mentos.make_variables( x, fullS, mu0, deltaG0, R, T )
    return entropy( np.concatenate( ( forward_probability, backward_probability ) ) )*net_flux[biomass]

def all_constraints( x, fullS, mu0, deltaG0, R, T  ):
    log_c, \
    forward_rate, backward_rate, \
    log_Q, log_K, \
    forward_likelihood, backward_likelihood, \
    forward_probability, backward_probability, mu, \
    thermodynamic_driving_force, \
    net_flux = mentos.make_variables( x, fullS, mu0, deltaG0, R, T )
    return np.concatenate((np.dot(S,forward_likelihood - backward_likelihood),
                            np.dot(S,net_flux),
                            np.log(forward_rate) - np.log(backward_rate) + log_Q - log_K,
                            [ log_c[A_ext] - np.log(c_U),  # = 0               # Creating a concentration gradient between A, E and D,F
                              log_c[E_ext] - np.log(c_U),  # = 0
                              log_c[D_ext] - np.log(c_L),  # = 0
                              log_c[F_ext] - np.log(c_L)],  #= 0
                            -abs(external_free_energy) - np.dot(fullS.T.as_matrix(),mu), # <= 0
                            np.dot(fullS.T.as_matrix(),mu) - abs(external_free_energy),  # <= 0 
                            np.minimum(-1, thermodynamic_driving_force)+ 1 -net_flux, # <= 0
                            net_flux - np.maximum(1, thermodynamic_driving_force ) - 1)
                           ) # <= 0

def thermo_and_boundary( x, fullS, mu0, deltaG0, R, T ):
    log_c, \
    forward_rate, backward_rate, \
    log_Q, log_K, \
    forward_likelihood, backward_likelihood, \
    forward_probability, backward_probability, mu, \
    thermodynamic_driving_force, \
    net_flux = mentos.make_variables( x, fullS, mu0, deltaG0, R, T )
    return np.concatenate((np.log(forward_rate) - np.log(backward_rate) + log_Q - log_K,
                           [  log_c[A_ext] - np.log(c_U),  # = 0               # Creating a concentration gradient between A, E and D,F
                              log_c[E_ext] - np.log(c_U),  # = 0
                              log_c[D_ext] - np.log(c_L),  # = 0
                              log_c[F_ext] - np.log(c_L)]))  #= 0)
def make_macro_maxent_production_rate_objective(fullS, S, mu0, deltaG0, R, T, c_L, c_U,v_L, v_U, objective_function, constraint_function):
    external_mets = [met for met in fullS.index if 'ext' in met]
    met_bounds = pd.Series({'A_ext':c_U, 'E_ext': c_U, 'F_ext': c_L, 'D_ext': c_L}, index=external_mets)
    mu_ext = mu0[external_mets] + R*T*met_bounds.apply(np.log)
    external_free_energy =  (mu_ext['D_ext'] + mu_ext['F_ext']) - (mu_ext['A_ext'] + mu_ext['E_ext'])
    
    def macro_maxent_production_rate(x):
        """Maximize product of  entropy production and macroscopic biomass growth rate of the ABC model"""
        f = objective_function( x, fullS, mu0, deltaG0, R, T  )
        g = constraint_function( x, fullS, mu0, deltaG0, R, T )
        fail = 0
        return -f, g, fail
    return macro_maxent_production_rate

In [4]:

variable_group = {'rates_and_log_c':[  
                    dict(name='log_c',nvars=m, type='c', initial_value=metab[convex_mentos]['Log Concentrations'].values),
                    dict(name='forward_rate', nvars=n, type='c', initial_value=reactions[convex_mentos]['Forward rate'].values),
                    dict(name='backward_rate',nvars=n, type='c', initial_value=reactions[convex_mentos]['Backward rate'].values)],
                 'likelihoods_and_log_c': [
                    dict(name='forward_likelihood', nvars=n, type='c', initial_value=reactions[convex_mentos]['Forward likelihoods'].values),
                    dict(name='log_c',nvars=m, type='c', initial_value=metab[convex_mentos]['Log Concentrations'].values)]}
       
constraint_group={'all_constraints':[
                    dict(name='steady_state_net_likelihood', ncons=i, type='e'),
                    dict(name='steady_state_flux', ncons=i, type='e'),
                    dict(name='thermodynamics',ncons=n, type='e'),
                    dict(name='boundary_conditions', ncons=m-i, type='e'),
                    dict(name='energy_barrier', ncons=n, type='i'),
                    dict(name='energy_sink',ncons=n,type='i'),
                    dict(name='flux_lower_bounds', ncons=n, type='i'),
                    dict(name='flux_upper_bounds', ncons=n,type='i')],
                  'thermo_and_boundary':[dict(name='thermodynamics',ncons=n, type='e'),
                                         dict(name='boundary_conditions', ncons=m-i, type='e'),],
                  'constraints_on_likelihoods': [dict(name='thermodynamics',ncons=n, type='e'),
                                                  dict(name='boundary_conditions', ncons=m-i, type='e'),]}
constraint_function = {'all_constraints': all_constraints,
                      'thermo_and_boundary': thermo_and_boundary,
                       'constraints_on_likelihoods': constraints_on_likelihoods
                      }
objective_function = {'macroscopic_entropy_production_rate': macroscopic_entropy_production_rate,
                      'macroscopic_entropy_production_rate_and_net_likelihood_ss':macroscopic_entropy_production_rate_and_net_likelihood_ss}

algorithms = [#('macroscopic_entropy_production_rate','rates_and_log_c', 'all_constraints' ), 
              #('macroscopic_entropy_production_rate','rates_and_log_c', 'thermo_and_boundary'), 
              ('macroscopic_entropy_production_rate_and_net_likelihood_ss','likelihoods_and_log_c', 'constraints_on_likelihoods')]

for og, vg, cg in algorithms:
    metab[cg], reactions[cg] = mentos.run_mentos_w_likelihoods(fullS, S, internal_mets, deltaG0, mu0, 
                                        variable_group[vg], constraint_group[cg],
                                        obj=make_macro_maxent_production_rate_objective( fullS, S, mu0, deltaG0, R, T, c_L, c_U,v_L, v_U, 
                                                                                         objective_function=objective_function[og],
                                                                                         constraint_function=constraint_function[cg]),
                                        biomass_rxn='R_8')
    mentos.print_report( os.path.join(abc_dir, cg), 'ABC_metab_{}.csv', metab[cg] )
    mentos.print_report( os.path.join(abc_dir, cg), 'ABC_reaction_{}.csv', reactions[cg])
    #display(Markdown('### ' + cg))
    #display(metab[cg].T)
                             
    

display(HTML(mentos.compare_frames(**metab).to_html()))

display(HTML(mentos.compare_frames(**reactions).to_html()))